In [1]:
from imports import *
from dataset import ToxicDataset
from model import LstmNet
from metrics.eval import ToxicEvaluation


In [2]:
# define Paths to datafile and embeddings
TOXIC_CSV_PATH = "../input/datasets/train.csv"
GLOVE_PATH = "../input/glove_embeddings/Embeddings/glove.6B.50d.txt"


In [3]:
# Initialize Dataset class
toxic_dataset = ToxicDataset(TOXIC_CSV_PATH, GLOVE_PATH)

using pre-built vocab


In [4]:
toxic_dataset.df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
batch_size = 64
nb_lstm_units = 64
random_seed = 42

val_split = 0.1
test_split = 0.1
shuffle_dataset = False

In [6]:
train_sampler, val_sampler, test_sampler = train_test_sampler(toxic_dataset, .8, .1, .1)

In [7]:
# Creating PT data samplers and loaders:

train_dataloader = DataLoader(toxic_dataset, batch_size, sampler=train_sampler)
val_dataloader = DataLoader(toxic_dataset, len(val_sampler.indices), sampler=val_sampler)
test_dataloader = DataLoader(toxic_dataset, len(test_sampler.indices), sampler=test_sampler)



In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
model = LstmNet(toxic_dataset.initial_embeddings, 200, device)

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
BCELoss = nn.BCELoss()
losses = {}
num_epochs = 1



In [18]:
for epoch in range(num_epochs):
    losses[epoch] = []
    for data_sample in train_dataloader:
        for i in range(len(data_sample)):

            data_sample[i] = data_sample[i].to(device)

        model.zero_grad()


        out = model(data_sample[0])
        loss = BCELoss(out, data_sample[1].float())

        loss.backward()
        optimizer.step()
        print(loss.data)
        losses[epoch].append(loss.data)




tensor(0.6918)
tensor(0.6698)
tensor(0.5921)
tensor(0.4025)


KeyboardInterrupt: 

In [15]:
idx.shape

torch.Size([64, 4521])

In [13]:


#   torch.save(model.state_dict(), ../output/'model' + datetime.now().date())
toxic_eval =ToxicEvaluation(model, val_dataloader, device)

NameError: name 'test_dataloader' is not defined

In [ ]:

for epoch in range(num_epochs):
    losses[epoch] = []
    for data_sample in tqdm(train_dataloader):
        
        model.zero_grad()
        idx = np.argsort(-data_sample[3])

        for i in range(len(data_sample)):
            data_sample[i] = data_sample[i][idx].to(device)

        out = declare(data_sample[0], data_sample[1], data_sample[2], data_sample[3], data_sample[4], data_sample[5])
        loss = BCEloss(out, data_sample[6].float())

        l2_reg = None
        for param in declare.named_parameters():
            if 'dense' in param[0] and 'weight' in param[0]:
                if l2_reg is None:
                    l2_reg = param[1].norm(2)
                else:
                    l2_reg = l2_reg + param[1].norm(2)

        total_loss = loss + reg_lambda*l2_reg

        total_loss.backward()
        optimizer.step()

        writer.add_scalar('total loss', total_loss, epoch)
        writer.add_scalar('regularization loss', l2_reg, epoch)
        writer.add_scalar('loss', loss, epoch)
        writer.add_scalar('pad embedding', declare.word_embeddings(torch.tensor([0]).to(device)).data.mean())
        
        losses[epoch].append(total_loss.data)

0
tensor([0., 0., 0., 0., 0., 0.])
1
tensor([1., 0., 0., 0., 1., 0.])
2
tensor([0., 0., 0., 0., 0., 0.])
